In [6]:
import kagglehub
import re

In [18]:
# Download latest version
path = kagglehub.dataset_download("abdallahalidev/plantvillage-dataset")

print("Path to dataset files:", path)
path = path + r"\plantvillage dataset" + r"\color"

Path to dataset files: C:\Users\herde\.cache\kagglehub\datasets\abdallahalidev\plantvillage-dataset\versions\3


In [19]:
import os
import shutil
import random

# Paths
original_dataset_dir = path  # Path to the original dataset
output_base_dir = 'split_dataset'  # Output directory for train, val, test

# Create train, val, test directories
splits = ['train', 'validation', 'test']
for split in splits:
    split_path = os.path.join(output_base_dir, split)
    os.makedirs(split_path, exist_ok=True)

# Split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Split images
for class_name in os.listdir(original_dataset_dir):
    class_path = os.path.join(original_dataset_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    # Create class directories in each split folder
    for split in splits:
        os.makedirs(os.path.join(output_base_dir, split, class_name), exist_ok=True)

    # Get all image files
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    random.shuffle(images)

    # Calculate split sizes
    total_images = len(images)
    train_size = int(total_images * train_ratio)
    val_size = int(total_images * val_ratio)

    # Assign images to splits
    train_images = images[:train_size]
    val_images = images[train_size:train_size + val_size]
    test_images = images[train_size + val_size:]

    # Function to copy images
    def copy_images(image_list, split):
        for image in image_list:
            src = os.path.join(class_path, image)
            dest = os.path.join(output_base_dir, split, class_name, image)
            shutil.copy(src, dest)

    # Copy images to respective folders
    copy_images(train_images, 'train')
    copy_images(val_images, 'validation')
    copy_images(test_images, 'test')

print("Dataset successfully split!")


Dataset successfully split!


In [12]:
# Extracts information from the label
def parse_label(label):
    # Split the label into the flower name and disease part
    label = label.split(" ")[0]
    parts = label.split("___")
    
    # Extract the flower name and make it human-readable
    flower_name = parts[0].replace("_", " ").replace("(", "").replace(")", "")
    
    # Determine if the label indicates a healthy plant
    is_healthy = "healthy" in label
    
    # Extract the disease name or mark it as healthy
    if is_healthy:
        disease = "healthy"
    else:
        disease = parts[1].replace("_", " ").replace("(", "").replace(")", "")
    
    return flower_name, is_healthy, disease

# Example usage

print(parse_label("Corn_(maize)___Northern_Leaf_Blight"))
print(parse_label("Cherry_(including_sour)___Powdery_mildew"))
print(parse_label("Blueberry___healthy"))
print(parse_label("Tomato___Spider_mites Two-spotted_spider_mite"))




('Corn maize', False, 'Northern Leaf Blight')
('Cherry including sour', False, 'Powdery mildew')
('Blueberry', True, 'healthy')
('Tomato', False, 'Spider mites')
